# Audio Compressor Golden Standard

Changing ```thresholdPercent``` and ```ratio``` allows one to adjust the peak threshold level and compression ratio

In [13]:
using WAV
using FFTW
using Plots
using Dates

## Reading in the WAV file and converting into dB
x, fs, bitlev, opt = WAV.wavread("StarWarsAudio.wav")

X = zeros(length(x))

for i = 1:length(x)
    if x[i] == 0
        X[i] = 0
    else
        X[i] = 20*log10(abs(x[i]))
    end
end

## Threshold Calcs
thresholdPercent = 0.30
ratio = 6
max = maximum(X)
thresholdVal = max*thresholdPercent

## Output Array Setup
Y = zeros(length(X))

ti = Dates.now() #start timer

## Audio Sample Compressor
for i = 1:length(X)
    if X[i] > thresholdVal
        Y[i] = (1/ratio)*(X[i] + thresholdVal) - thresholdVal
    else
        Y[i] = X[i]
    end
end

## Reconstruct Audio
for i = 1:length(X)
    if x[i] == 0
        Y[i] = 0
    elseif x[i] < 0
        Y[i] = -1*(10^(Y[i]/20))
    else
        Y[i] = 10^(Y[i]/20)
    end
end

te = Dates.now() #stop timer

print("Old Maximum Sample: ", round(maximum(abs.(x)), digits = 3))
print("\nNew Maximum Sample: ", round(maximum(abs.(Y)), digits = 3))
# print("\nAverage Reduction: ", round(maximum(abs.(Y))/maximum(abs.(x))*100, digits = 2), "%")

WAV.wavwrite(Y, "compressed.wav", Fs=fs)

F = fft(Y)
freqs = fftfreq(length(Y), 48000)
plot(freqs, abs.(F), title = "Spectrum", xlim=(-1000, +1000))

print("\nfunction time = ", te-ti) #time taken to compress audio

Old Maximum Sample: 0.344
New Maximum Sample: 0.344
function time = 244 milliseconds

# References:

https://www.audiolabs-erlangen.de/resources/MIR/FMP/C3/C3S1_LogCompression.html

https://jstage.jst.go.jp/article/ast/29/6/29_6_372/_pdf/-char/en

https://docs.juliahub.com/WAV/bHvvS/1.1.0/#WAV.wavwrite